<h1>Excel Worker - LangCahin ReaAct Agent</h1>

In [ ]:
%pip install langchain langgraph pandas python-dotenv duckdb

In [ ]:
import pandas as pd
import duckdb
import json
from typing import TypedDict, Optional, List, Dict, Union
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from IPython.display import display

import os

from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
# State management
def create_agent_state(file_name: str, user_query: str) -> dict:
    return {
        "file_name": file_name,
        "user_query": user_query,
        "messages": [],
        "preview_data": None,
        "last_query": None,
        "query_result": None,
        "error": None,
    }

def print_state(state: dict, title: str = "Current State"):
    """Print current state of the agent"""
    print(f"\n{'='*20} {title} {'='*20}")
    print(f"File: {state['file_name']}")
    print(f"User Query: {state['user_query']}")
    
    if state['error']:
        print(f"\n🔴 Error: {state['error']}")
        
    if state['last_query']:
        print("\n📝 Last Query:")
        print(state['last_query'])
        
    if state['query_result']:
        print("\n✅ Query Result:")
        if isinstance(state['query_result'], dict) and 'rows' in state['query_result']:
            df = pd.DataFrame(state['query_result']['rows'])
            display(df)
        else:
            print(json.dumps(state['query_result'], indent=2))
    print(f"\n{'='*50}\n")

def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Robust preprocessing of DataFrame to handle empty values."""
    # Convert empty strings and whitespace to None
    df = df.replace(r'^\s*$', None, regex=True)
    # Convert NaN strings to None
    df = df.replace(['nan', 'NaN', 'null'], None)
    # Convert pandas NaN to None
    df = df.where(pd.notnull(df), None)
    return df

# Tool definitions with single string input
@tool
def preview_excel_structure(input_str: str) -> str:
    """Use this first to examine the Excel file structure and data types. 
    The input should be a JSON string with format: {"file_name": "your_file.xlsx"}"""
    try:
        data = json.loads(input_str)
        file_name = data.get("file_name")
        if not file_name:
            return json.dumps({"error": "File name must be provided"})
            
        df = pd.read_excel(file_name)
        df = preprocess_dataframe(df)  # Apply preprocessing
        df_sample = df.head(3).astype(str)
        print("✅ Preview successful")
        display(df_sample)
        return json.dumps({
            "result": {
                "columns": df.columns.tolist(),
                "dtypes": df.dtypes.astype(str).to_dict(),
                "sample_rows": df_sample.to_dict(orient="records")
            }
        })
    except Exception as e:
        return json.dumps({"error": str(e)})
        
@tool
def complex_duckdb_query(input_str: str) -> str:
    """Use this for complex SQL operations (GROUP BY, aggregations, etc.).
    The input should be a JSON string with format: 
    {"file_name": "your_file.xlsx", "query": "your SQL query"}
    Note: Use 'data' as the table name in your SQL queries."""
    try:
        data = json.loads(input_str)
        file_name = data.get("file_name")
        query = data.get("query")
        if not file_name or not query:
            return json.dumps({"error": "Both file_name and query must be provided"})

        print("\n🔍 Executing DuckDB query:")
        print(query)
        
        # Read and preprocess the Excel file to handle empty values
        df = pd.read_excel(file_name)
        df = preprocess_dataframe(df)  # Clean the data upfront
        
        with duckdb.connect() as con:
            # Register dataframe with a consistent table name 'data'
            con.register("data", df)
            result = con.execute(query).fetchdf()
            print("✅ Query successful")
            display(result)
            
            # Handle different types of results robustly
            if result is None:
                return json.dumps({"result": None})
                
            if isinstance(result, pd.DataFrame):
                # Handle DataFrame results
                try:
                    # Replace various null/nan types with None
                    df_processed = result.copy()
                    
                    # Handle infinite values
                    df_processed = df_processed.replace([float('inf'), -float('inf')], None)
                    
                    # Replace numpy NaN, pandas NA, and other null types with None
                    df_processed = df_processed.where(pd.notna(df_processed), None)
                    
                    # Ensure all values are JSON serializable
                    for column in df_processed.columns:
                        if df_processed[column].dtype == 'object':
                            df_processed[column] = df_processed[column].apply(
                                lambda x: str(x) if x is not None else None
                            )
                    
                    return json.dumps({
                        "result": {
                            "columns": df_processed.columns.tolist(),
                            "rows": df_processed.to_dict(orient="records")
                        }
                    })
                except Exception as e:
                    return json.dumps({
                        "error": f"Error processing DataFrame: {str(e)}",
                        "result": result.to_string()  # Fallback to string representation
                    })
            else:
                # Handle scalar or other types of results
                try:
                    return json.dumps({"result": result})
                except TypeError:
                    return json.dumps({
                        "result": str(result)  # Fallback to string representation
                    })
            
    except Exception as e:
        return json.dumps({"error": str(e)})

@tool
def simple_dataframe_query(input_str: str) -> str:
    """Use this for simple Pandas operations on the dataframe.
    The input should be a JSON string with format:
    {"file_name": "your_file.xlsx", "query": "your pandas expression"}"""
    try:
        data = json.loads(input_str)
        file_name = data.get("file_name")
        query = data.get("query")
        if not file_name or not query:
            return json.dumps({"error": "Both file_name and query must be provided"})

        print("\n🔍 Executing Pandas query:")
        print(query)
        
        df = pd.read_excel(file_name)
        df = preprocess_dataframe(df)  # Apply preprocessing
        result = eval(query, {"df": df, "pd": pd})
        
        if isinstance(result, pd.DataFrame):
            output = {
                "result": {
                    "columns": result.columns.tolist(),
                    "rows": result.fillna(None).to_dict(orient="records")
                }
            }
        else:
            output = {"result": str(result)}
        
        print("✅ Query successful")
        if isinstance(result, pd.DataFrame):
            display(result)
        else:
            print(result)
            
        return json.dumps(output)
    except Exception as e:
        return json.dumps({"error": str(e)})


# Tools list
tools = [preview_excel_structure, complex_duckdb_query, simple_dataframe_query]

# System prompt
SYSTEM_PROMPT = """You are an Excel analysis expert that helps users analyze Excel files using SQL and Pandas.

IMPORTANT WORKFLOW:
1. ALWAYS start by using preview_excel_structure to understand the data structure
2. Based on the preview, choose the appropriate tool:
   - For grouping, aggregations, or complex calculations: use complex_duckdb_query
   - For simple row operations or basic calculations: use simple_dataframe_query

For DuckDB SQL queries, ALWAYS:
1. Handle empty cells in calculations:
   - For sums and averages, count only non-null values
   - When calculating averages across multiple columns:
     * Sum only non-null values from each column
     * Divide by the count of non-null values
   * For complex calculations, use WITH clauses to split steps, example: WITH individual_calcs AS (SELECT id, CASE WHEN "Value" IS NOT NULL THEN "Value" END as clean_value FROM data), group_calcs AS (SELECT group_col, SUM(clean_value) / NULLIF(COUNT(clean_value), 0) as avg_value FROM individual_calcs GROUP BY group_col) SELECT * FROM group_calcs

2. Cast strings to proper types:
   CAST(column AS DOUBLE)

3. For numeric ranges use:
   CASE WHEN col >= 0 AND col < 5 THEN '0-5' WHEN col >= 5 AND col < 10 THEN '5-10' END as range

4. Include all non-aggregated columns in GROUP BY

5. Use double quotes for columns with spaces: "Column Name"

6. For complex aggregations:
   - First calculate individual metrics
   - Then group and aggregate
   - Finally format output
   Example query: WITH symbol_metrics AS (SELECT "Symbol", (COALESCE("Month1", 0) + COALESCE("Month2", 0)) / NULLIF(CAST("Month1" IS NOT NULL AS INTEGER) + CAST("Month2" IS NOT NULL AS INTEGER), 0) as avg_performance FROM data), group_metrics AS (SELECT group_col, AVG(avg_performance) as group_performance, STRING_AGG(Symbol, ', ') as symbols_in_group FROM symbol_metrics GROUP BY group_col) SELECT * FROM group_metrics

For Pandas queries:
1. Reference the dataframe as 'df'
2. For ranges use: pd.cut(df['column'], bins=[0,5,10], labels=['0-5','5-10'])
3. For aggregations use: df.groupby().agg()

IMPORTANT PATTERNS FOR COMPLEX ANALYSIS:
1. Calculate individual metrics first:
   - Handle NULL values with COALESCE
   - Count valid values using CAST(IS NOT NULL AS INTEGER)
2. Group results:
   - Use proper group by
   - Include supporting information
3. Format final output:
   - Order results if needed
   - Include all requested information

IMPORTANT: All tool inputs must be properly formatted JSON strings. Examples:

1. Preview Excel:
{"file_name": "example.xlsx"}

2. DuckDB Query with proper empty value handling:
{"file_name": "example.xlsx", "query": "WITH metrics AS (SELECT CASE WHEN price >= 0 AND price < 5 THEN '0-5' ELSE '5+' END as range, value FROM data WHERE value IS NOT NULL) SELECT range, AVG(value) as avg_val FROM metrics GROUP BY range"}

3. Pandas Query:
{"file_name": "example.xlsx", "query": "df.groupby(pd.cut(df['col'], bins=[0,5,10])).agg(['mean', 'count'])"}

Remember:
1. Tool inputs must be valid JSON strings
2. Preview data first
3. Always exclude empty/null values from calculations
4. Use proper column references
5. Queries must be on a single line (no line breaks in JSON strings)
6. For complex analyses:
   - Break calculations into steps using WITH clauses
   - Calculate individual metrics before grouping
   - Include supporting information (counts, lists) in output"""


# Agent setup remains the same
def setup_agent(debug=True):
    """Initialize the ReAct agent."""
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    # Create prompt template with required variables
    prompt = PromptTemplate.from_template(
        template="""Answer the following questions as best you can.

{system_prompt}

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}""")
    
    # Get tool names for the template
    tool_names = [tool.name for tool in tools]
    
    # Create the ReAct agent
    agent = create_react_agent(
        llm=llm,
        tools=tools,
        prompt=prompt
    )
    
    return AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=debug,
        max_iterations=3,
        handle_parsing_errors=True
    )

def process_excel_query(file_name: str, user_query: str, debug: bool = True) -> dict:
    """Process a query about an Excel file using the ReAct agent with state tracking."""
    # Initialize state
    state = create_agent_state(file_name, user_query)
    
    try:
        agent = setup_agent(debug=debug)
        input_query = f"{user_query} Using the file: {file_name}"
        
        if debug:
            print(f"\n🎬 Starting Agent Execution\nQuery: {input_query}")
            print_state(state, "Initial State")
        
        # Update state before execution
        # state['attempts'] += 1
        
        # Pass both input and system_prompt
        result = agent.invoke({
            "input": input_query,
            "system_prompt": SYSTEM_PROMPT
        })
        
        # Update state after execution
        intermediate_steps = result.get('intermediate_steps', [])
        if intermediate_steps:
            # Store the last action and input
            last_step = intermediate_steps[-1]
            try:
                tool_info = {
                    "tool": last_step[0].tool,
                    "input": last_step[0].tool_input
                }
                state['last_query'] = json.dumps(tool_info, indent=2)
                
                # Try to parse the result
                step_result = last_step[1]
                if isinstance(step_result, str):
                    try:
                        state['query_result'] = json.loads(step_result)
                    except json.JSONDecodeError:
                        state['query_result'] = {"result": step_result}
                else:
                    state['query_result'] = {"result": str(step_result)}
                
                # Store any error messages
                if isinstance(state['query_result'], dict):
                    if 'error' in state['query_result']:
                        state['error'] = state['query_result']['error']
                    elif 'result' in state['query_result']:
                        state['error'] = None  # Clear any previous errors
            except Exception as e:
                state['error'] = f"Error processing state: {str(e)}"
                state['query_result'] = {"error": str(e)}
        
        if debug:
            print("\n✅ Agent Execution Completed")
            print_state(state, "Final State")
        
        return {
            "success": True,
            "output": result['output'],
            "steps": intermediate_steps,
            "state": state
        }
        
    except Exception as e:
        state['error'] = str(e)
        if debug:
            print(f"\n❌ Agent Execution Failed: {str(e)}")
            print_state(state, "Error State")
        return {
            "success": False,
            "error": str(e),
            "state": state
        }

In [ ]:
# Process your query
result = process_excel_query(
    file_name="ipo_data.xlsx",
    user_query="What percentage of the proposedTickerSymbol-s whit proposedSharePrice between 0 and 15.00 have average positive performance above 10%. The performance in percentage of each ticker is available in the months 1 through 13. Dont count empty months in the calculation.",
    debug=True
)


# Access state if needed
final_state = result['state']
print_state(final_state, "Final State Review")

# Group the proposedTickerSymbol symbols by increment of 5 based on proposedSharePrice. For each group take all proposedTickerSymbols in the group and calculate the average performance of each symbol. The performance of a symbol is spread in the months 1 through 13. Pay attention to empty value months, just ignore them from the calculation and reduce the month count in this case. Once you calculate the performance of each proposedTickerSymbol then calculate the peformance of the group. I would like to see the top 3 groups by performance and the proposedTickerSymbol in them.
# What percentage of the proposedTickerSymbol-s whit proposedSharePrice between 0 and 15.00 have average positive performance above 10%. The performance in percentage of each ticker is available in the months 1 through 13. Dont count empty months in the calculation.

